In [ ]:
#import kagglehub
#gunavenkatdoddi_eye_diseases_classification_path = kagglehub.dataset_download('gunavenkatdoddi/eye-diseases-classification')

#print('Data source import complete.')

In [ ]:
from img2vec_pytorch import Img2Vec
import os
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
img2Vec = Img2Vec()

In [ ]:
#img = Image.open("./dataset/Train/cataract/_0_4015166.jpg")
#print(img2Vec.get_vec(img))



In [ ]:
data_dir = "./dataset"
train_dir = os.path.join(data_dir,"Train")  
test_dir = os.path.join(data_dir,"Test")

data = {}
for i,dir_ in enumerate([train_dir,test_dir]):
    print(dir_)
    features = []
    labels = []

    for category in os.listdir(train_dir):
        for img_name in os.listdir(os.path.join(dir_,category)):
            img_path = os.path.join(dir_,category,img_name)
            img = Image.open(img_path)
            img_features = img2Vec.get_vec(img)

            features.append(img_features)
            labels.append(category)
            
    data[['train_data','test_data'][i]] = features
    data[['train_label','test_label'][i]] = labels
print(data)
    


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data['train_data'] = scaler.fit_transform(data['train_data'])
data['test_data'] = scaler.transform(data['test_data'])



In [ ]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
model = RandomForestClassifier()
model.fit(data['train_data'],data['train_label'])
y_pred = model.predict(data['test_data'])
score = accuracy_score(y_pred ,data['test_label'])
print(score)
report = classification_report(data['test_label'], y_pred)
print("Classification Report:\n", report)
confusion = confusion_matrix(data['test_label'], y_pred)
print(confusion)

In [ ]:
with open('./eyesDiseaseModel.p','wb') as f:
    pickle.dump(model,f)
    f.close()

In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(data['train_data'],data['train_label'])
y_pred = model.predict(data['test_data'])
score = accuracy_score(y_pred ,data['test_label'])
print(score)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['train_label'] = le.fit_transform(data['train_label'])
data['test_label'] = le.transform(data['test_label'])


In [ ]:
import numpy as np

data['train_data'] = np.array(data['train_data'])
data['train_label'] = np.array(data['train_label'])
print(data['train_label'])


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', verbosity=0)
model.fit(data['train_data'],data['train_label'])
y_pred = model.predict(data['test_data'])
score = accuracy_score(y_pred ,data['test_label'])
print(score)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svm', SVC(probability=True))
], voting='soft')
ensemble.fit(data['train_data'], data['train_label'])
y_pred = ensemble.predict(data['test_data'])
score = accuracy_score(y_pred ,data['test_label'])
print(score)


In [ ]:
# with open('./eyesDiseaseModel.p','wb') as f:
#     pickle.dump(model,f)
#     f.close()